In [ ]:
[전처리 순서 (미완)]

1. 다운받은 영상 데이터 불러오기
2. 영상 데이터(.mp4)를 음성 데이터(.wav)로 전환시킨 후에 따로 저장하기 -> data_1
3. data_1 파일 텍스트로 변환시켜서 단어 단위로 컷팅하고, 컷팅한 위치를 리스트 안에 저장하기
4. 영상 데이터에서 소리 제거한 후에 따로 저장하기 -> data_2
5. 음성 컷팅 위치와 data2를 대응시킨 후에 그 부분을 컷팅하여 따로 저장하기
    -> data_3
6. data_3 파일에서 하관/입술 부분만 크롭한 형태로 따로 저장하기 -> data_4

## 1. 영상 데이터 불러오기

In [1]:
import os
import re

In [2]:
# [필수] 파일 주소 재설정 
MOV_Directory = '../1. Crawling/MOV' 
VideoList = os.listdir(MOV_Directory)

## 2. 영상 파일을 음성 파일로 전환

In [3]:
# !pip install moviepy
import moviepy.editor as mp
from pydub import AudioSegment

In [4]:
# [필수] 파일 주소 재설정 
WAV_Directory = './WAV_streo' 

for i in VideoList:
    i2 = MOV_Directory + '/' + i
    clip = mp.VideoFileClip(i2)
    
    sound_file_name = re.sub(pattern='mp4', repl='wav', string=i)
    sound_file_name = WAV_Directory + '/' + sound_file_name
    clip.audio.write_audiofile(sound_file_name)

chunk:   3%|██                                                            | 86/2550 [00:00<00:02, 859.81it/s, now=None]

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003463.wav


MoviePy - Done.


chunk:   2%|█▎                                                            | 62/2868 [00:00<00:04, 619.87it/s, now=None]

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003464.wav


...

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003468.wav


MoviePy - Done.


chunk:   0%|                                                                        | 0/3178 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in ./WAV_streo/readnaversid1355oid056aid0000003469.wav


KeyboardInterrupt: 

In [5]:
WAV_stereo_List = os.listdir(WAV_Directory)

for i in WAV_stereo_List:
    i2 = WAV_Directory + '/' + i
    sound = AudioSegment.from_wav(i2)
    sound = sound.set_channels(1)
    
    mono_file_name = './WAV_mono/' + i  
    sound.export(mono_file_name, format="wav")

## 3. 음성 데이터 : 단어 기준 Tokenizing 시간 좌표 구하기

In [6]:
# 참고 링크 : https://cloud.google.com/speech-to-text/docs/quickstart-client-libraries#client-libraries-install-python

from google.cloud import speech

In [7]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    from google.cloud import speech

    client = speech.SpeechClient()

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        language_code="ko-KR",
        enable_word_time_offsets=True
    )

    operation = client.long_running_recognize(config=config, audio=audio)

    print("Waiting for operation to complete...")
    result = operation.result(timeout=90)

    for result in result.results:
        alternative = result.alternatives[0]
        print("Transcript: {}".format(alternative.transcript))
        print("Confidence: {}".format(alternative.confidence))

        for word_info in alternative.words:
            word = word_info.word
            start_time = word_info.start_time
            end_time = word_info.end_time

            print(
                f"Word: {word}, start_time: {start_time.total_seconds()}, end_time: {end_time.total_seconds()}"
            )

In [73]:
transcribe_gcs('gs://audionews/readnaversid1355oid056aid0000003463.wav')

Waiting for operation to complete...
Transcript: 왜 이런 증시 폭락이 왔을까요 미국 등 선진국들이 돈 풀기를 끝내는 이른바 출구전략을 쓰고 있기 때문입니다 일하던 출구전략이 본격화되면 세계 경제 아시아와 우리 경제에 큰 충격에 올 것이라는게 더 큰 문제입니다이어서 2홀 기자가 분석했습니다
Confidence: 0.8607030510902405
Word: 왜, start_time: 0.0, end_time: 1.1
Word: 이런, start_time: 1.1, end_time: 1.4
Word: 증시, start_time: 1.4, end_time: 1.8
Word: 폭락, start_time: 1.8, end_time: 2.2
Word: 이, start_time: 2.2, end_time: 2.3
Word: 왔을까요, start_time: 2.3, end_time: 2.9
Word: 미국, start_time: 2.9, end_time: 4.0
Word: 등, start_time: 4.0, end_time: 4.2
Word: 선진국들이, start_time: 4.2, end_time: 5.0
Word: 돈, start_time: 5.0, end_time: 5.4
Word: 풀기를, start_time: 5.4, end_time: 5.7
Word: 끝내는, start_time: 5.7, end_time: 6.1
Word: 이른바, start_time: 6.1, end_time: 6.4
Word: 출구전략을, start_time: 6.4, end_time: 7.2
Word: 쓰고, start_time: 7.2, end_time: 7.5
Word: 있기, start_time: 7.5, end_time: 7.6
Word: 때문입니다, start_time: 7.6, end_time: 8.0
Word: 일하던, start_time: 8.0, end_time: 9.1
Word: 출구전략이, start_time: 9.1, end_ti

In [ ]:
시간 좌표 구하기 성공! 
단, 이후의 작업을 위해서 시간 좌표를 튜플 or 리스트 형태로 변환시켜야 함.

단점1 : 생각보다 파일 하나 변환시키는 시간이 오래 걸림.
단점2 : 구글 STT api 환경 설정할 때 생각보다 시간이 꽤 걸림. 
-> 개인적으로 공식 구글 가이드 좀 불친절하다고 생각함...^^;;
-> https://www.youtube.com/watch?v=Ds-7D8d-FwA  [참고하면 좋은 영상]
-> https://cloud.google.com/storage/docs/quickstarts?hl=ko  [공식 구글 가이드]
    
단점3 : 1달 기준 60분 분량의 음성 데이터까지만 무료라서 하루에 돌릴 수 있는 양이 한정되어 있음.
-> 앵커 영상만 분리해서 불필요한 데이터를 제거한다면 이 핸디캡을 극복할 수 있지 않을까...?
-> 필요한 영상 데이터만 쏙쏙 뽑은 후, 각자 나눠서 돌린다면 현실 타협 가능성이 보이지 않을까?


장점1 : 단어 나누는 기준의 정확도는 높은 편이라고 생각함. 
-> 물론 중간에 틀린 단어들도 좀 있어서 필요하다면 이건 손을 볼 필요가 있다고 여겨짐.


만약 이 방법으로 진행한다면? 
-> 파일 n개를 일괄적으로 처리하는 코드 짜기 (이건 파일 1개만 사용했음)
-> 그리고 앞에서 얻은 시간 좌표 데이터를 토대로 소리 제거한 동영상을 컷팅할 생각임.
-> 태희가 말한 것처럼 앵커 아닌 영상 지우는 작업도 해야 함.


*** 혹시 모르니까 완전 무료 오픈소스도 찾아보겠음...! 

## 태희 코드 (nonsilent한 단위로 영상 컷팅)

In [75]:
# 확인 완료했다는 의미로 올렸습니다!

import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import re
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip


#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

#Convert wav to audio_segment
audio_segment = AudioSegment.from_wav(sound_file_name)

#normalize audio_segment to -20dBFS 
normalized_sound = match_target_amplitude(audio_segment, -20.0)
print("length of audio_segment={} seconds".format(len(normalized_sound)/1000))

#Print detected non-silent chunks, which in our case would be spoken words.
nonsilent_data = detect_nonsilent(normalized_sound, min_silence_len=500, silence_thresh=-20, seek_step=1)

a = 1
#convert ms to seconds
print("start,Stop")
for chunks in nonsilent_data:
    time_line = [[chunk/1000 for chunk in chunks]]
    for i in time_line:
        print(i[0], i[1])
        
        b = str(a)
        ffmpeg_extract_subclip("video.mp4", i[0], i[1], targetname="video"+b+".mp4")
        a += 1

#1, 단어별이나 문장별로 안 나온다. amplitude와 min_silence_len을 조절해볼 것
#2, 앵커가 말하는 부분만을 자르지 않았다. -> 이 문제는 상의해야 함
#   아직 앵커가 말하고 있음에도 불구하고, 화면 전환이 일어나며 얼굴이 나타나지 않음
#   이건 큰 화면 전환을 기준으로 잘라야 하는 문제일듯
#3, 이 코드들을 함수로 만들어 사용할 것 - 작은 문제

length of audio_segment=58.94 seconds


chunk:  45%|█████████████████████████▊                               | 1301/2868 [7:27:09<00:00, 1664.12it/s, now=None]

start,Stop
1.9 1.985
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
2.735 2.987
Moviepy - Running:
>>> "+ " ".join(cmd)


chunk:  45%|█████████████████████████▊                               | 1301/2868 [7:27:10<00:00, 1664.12it/s, now=None]

Moviepy - Command successful
3.538 4.616
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
8.092 8.097
Moviepy - Running:
>>> "+ " ".join(cmd)


...

Moviepy - Command successful
53.636 53.824
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
54.415 55.67
Moviepy - Running:
>>> "+ " ".join(cmd)


chunk:  45%|█████████████████████████▊                               | 1301/2868 [7:27:12<00:00, 1664.12it/s, now=None]

Moviepy - Command successful
57.492 57.95
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
58.563 58.94
Moviepy - Running:
>>> "+ " ".join(cmd)


chunk:  45%|█████████████████████████▊                               | 1301/2868 [7:27:12<00:00, 1664.12it/s, now=None]

Moviepy - Command successful
